In [ ]:
import subprocess, os
from os import listdir
import pandas as pd
import numpy as np
from os.path import exists

In [ ]:
import pubmed_parser as pp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import textstat
from matplotlib.colors import is_color_like

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
stop = set(stopwords.words('english') + list(string.punctuation))
alphabet = list(string.ascii_lowercase)

In [ ]:
def get_yolo_result(result):
    ans = []
    
    for i in range(len(result)-1):
        if 'Predicted in' in result[i]:
            file = result[i].split(': ')[0].split('/')[-1]
            if 'Predicted in' in result[i+1]:
                ans.append([file, 0])
            else:
                for j in range(2, 20):
                    if  i+j < len(result)-1 and 'Predicted in' in result[i+j]:
                        idx = j
                        break
                    elif i+j >= len(result)-1:
                        idx = j
                        break
                conf = 0
                for line in result[i+1:i+j]:
                    conf = max(conf, int(line.split(': ')[1].split('%')[0]))
                ans.append([file, conf])
    if 'Predicted in' in result[-1]:
        ans.append([result[-1].split(': ')[0].split('/')[-1], 0])
        
    return ans

In [ ]:
xml_path = '//'

In [ ]:
def caption(pmcid):
    tmp = [x for x in listdir(xml_path+pmcid+'/') if x[-5:] == '.nxml']
    if len(tmp) == 1:
        nxml_path = xml_path+pmcid+'/'+tmp[0]
    else:
        print(len(tmp))
        print(pmcid)
    return pp.parse_pubmed_caption(nxml_path)

In [ ]:
def check_float(s):
    try:
        float(s)
    except:
        return(s)

In [ ]:
symbol = ['trinagle', 'circle', 'square', 'diamond', 'trinagles', 'circles', 'squares', 'diamonds', '▪', '□', 
          'dashed', 'hatched']

In [ ]:
def color_count(text_list):
    c = 0
    for i in range(len(text_list)):
        if is_color_like(check_float(text_list[i])) or text_list[i] in symbol:
            c += 1
        if text_list[-1] != 'solid':
            if text_list[i] == 'solid' and text_list[i+1] in ['line', 'lines']:
                c += 1
    return c

In [ ]:
def plot(path):
    img = mpimg.imread(path)
    imgplot = plt.imshow(img)
    plt.figure(figsize=(15, 10))
    plt.show()

## Legend detection on subplots

In [ ]:
darknet = './darknet'
current = os.path.abspath(os.getcwd())
voc = current+'/data/legend.data'
cfg = current+'/data/yolov4_legend.cfg'
weights = current+'/yolov4_legend_last.weights'
subplot_list = current+'/yolo_detection_all/subplot_list.txt'
subplot_result = current+'/yolo_detection_all/subplot_legend_result.txt'

In [ ]:
# write figure path
subplot_folder_path = './line_charts/all/'
figure_path = listdir(subplot_folder_path)
f = open(subplot_list):
for ff in figure_path:
    f.write(subplot_folder_path+ff+'\n')
f.close()

In [ ]:
subplot_legend_command = darknet+' detector test '+voc+' '+cfg+' '+weights+' '+ \
        '-dont_show -ext_output < ' +subplot_list+ ' > '+subplot_result

In [ ]:
p = subprocess.Popen(subplot_legend_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, 
                     cwd='./darknet', shell=True)

In [ ]:
p.communicate()

In [ ]:
# Getting result and wirte into dataframe
f = open(subplot_result, "r")
subplot_legend_detection = f.readlines()
f.close()

In [ ]:
subplot_legend_detection = [x for x in subplot_legend_detection if 'Predicted in' in x or 'Legend' in x]

In [ ]:
df = pd.DataFrame(get_yolo_result(subplot_legend_detection), columns = ['File', 'Legend'])

## Legend detection on Compound (Original) figures

In [ ]:
# get corresponding compound figures of subplots
compound = []
compound_path = '//'
for f in figure_path:
    if f[-6] == '_':
        if exists(compound_path+f[:-6]+'.png'):
            compound.append(compound_path+f[:-6]+'.png')
        elif exists(compound_path+f[:-6]+'.jpg'):
            compound.append(compound_path+f[:-6]+'.jpg')
    elif f[-7] == '_':
        if exists(compound_path+f[:-7+'.png'):
            compound.append(compound_path+f[:-7]+'.png')
        elif exists(compound_path+f[:-7]+'.jpg'):
            compound.append(compound_path+f[:-7]+'.jpg')      

In [ ]:
darknet = './darknet'
current = os.path.abspath(os.getcwd())
voc = current+'/data/legend.data'
cfg = current+'/data/yolov4_legend.cfg'
weights = current+'/yolov4_legend_last.weights'
compound_list = current+'/yolo_detection_all/compound_list.txt'
compound_result = current+'/yolo_detection_all/compound_result.txt'

In [ ]:
f = open(compound_list, "w")
for ff in compound:
    f.write(ff+'\n')
f.close()

In [ ]:
compound_legend_command = darknet+' detector test '+voc+' '+cfg+' '+weights+' '+ \
        '-dont_show -ext_output < ' +compound_list+ ' > '+compound_result

In [ ]:
p = subprocess.Popen(compound_legend_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, 
                     cwd='./darknet', shell=True)

In [ ]:
p.communicate()

In [ ]:
f = open(compound_result, "r")
compound_legend_detection = f.readlines()
f.close()

In [ ]:
compound_legend_detection=[x for x in compound_legend_detection if 'Predicted in' in x or 'Legend' in x]
compound_dict = dict(get_yolo_result(compound_legend_detection))

In [ ]:
## Combine subplot and compound legend detection result
for i in range(len(df)):
    tmp_f = df.loc[i, 'File']
    if tmp_f[:-6] + '.png' in compound_dict:
        df.loc[i, 'Original'] = compound_dict[tmp_f[:-6] + '.png']
    elif tmp_f[:-6] + '.jpg' in compound_dict:
        df.loc[i, 'Original'] = compound_dict[tmp_f[:-6] + '.jpg']
    elif tmp_f[:-7] + '.png' in compound_dict:
        df.loc[i, 'Original'] = compound_dict[tmp_f[:-7] + '.png']
    elif tmp_f[:-7] + '.jpg' in compound_dict:
        df.loc[i, 'Original'] = compound_dict[tmp_f[:-7] + '.jpg']
    else:
        df.loc[i, 'Original'] = -1
        print(tmp_f)

## Classify if a graph need legend

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory('./line_charts/', batch_size=4, image_size=(224,224))
file_name = [x.split('/')[-1] for x in dataset.file_paths]

In [ ]:
legend_classifier = tf.keras.models.load_model('need_legend_classifier/')

In [ ]:
predictions = legend_classifier.predict(dataset)
predictions = tf.nn.sigmoid(predictions)
predictions = np.array(tf.where(predictions < 0.5, 0, 1))
pred_list = [x[0] for x in predictions]

In [ ]:
need_legend_df = pd.DataFrame({'File':file_name, 'need_legend':pred_list})
df = df.merge(need_legend_df, left_on='File', right_on='File')

## Caption Analysis

In [ ]:
color_names, color_rgb = primary_colors('Base Colors')

In [ ]:
for j in range(len(df)):
    f = df.loc[j, 'File']
    # caption analysis
    pmcid = f.split('_')[0]
    try:
        tmp_caption = caption(pmcid)
        for i in range(len(tmp_caption)):
            if tmp_caption[i]['graphic_ref'] == '_'.join(f.split('_')[1:-1]) or \
                tmp_caption[i]['graphic_ref'] == '_'.join(f.split('_')[1:])[:-4]:
                text = tmp_caption[i]['fig_caption']
                text_index = textstat.automated_readability_index(text)
                text_list = [i for i in word_tokenize(text.lower()) if i not in stop and i not in alphabet]
                color_index = color_count(text_list)
                df.loc[j, ['read_index', 'explain_count']] = [text_index, color_index]
                
    except:
        df.loc[j, ['read_index', 'explain_count']] = [-1, -1]
        print(pmcid)

## Final Result

In [ ]:
df